# Agents and RAG, A Technical Deep Dive 

In this notebook, i'll be using the Lang and Llama family for building and exploring RAG from scratch and the techniques we can do with Agents

### Brief History

The concept of intelligent agents has evolved dramatically over the past seven decades, transforming from simple rule-based systems to today's sophisticated AI companions that can reason, plan, and act autonomously. Understanding this progression is essential because it helps us appreciate why modern agentic systems represent such a significant breakthrough and why they're becoming central to how we build AI applications. The journey began in the 1950s when researchers like Allen Newell and Herbert Simon created the Logic Theorist, a program that could prove mathematical theorems by exploring different logical paths. These early agents were like skilled craftsmen—they could perform specific tasks very well, but only within narrow, pre-defined domains. The 1970s and 1980s brought expert systems like MYCIN for medical diagnosis and DENDRAL for chemical analysis. While impressive, these systems required months of manual knowledge engineering, where human experts had to explicitly encode their domain knowledge into rigid rule sets.

The 1990s marked a shift toward more flexible software agents that could operate in networked environments and coordinate with other agents. This period introduced the concept of multi-agent systems, where multiple specialized agents could collaborate to solve complex problems. However, these systems still required extensive manual programming and could only handle situations their creators had anticipated. The real transformation began in the 2000s with machine learning advances. Agents could now learn from data rather than relying solely on hand-coded rules. Virtual assistants like Siri and Alexa brought agent technology to mainstream consumers, though they remained relatively narrow in scope—essentially sophisticated voice interfaces for search and simple task execution.

<img src="https://miro.medium.com/1*Ygen57Qiyrc8DXAFsjZLNA.gif" width=700>

The breakthrough moment arrived with large language models starting around 2020. Systems like GPT-3 and GPT-4 combined vast knowledge with sophisticated reasoning abilities, creating agents that could understand natural language, maintain context across conversations, and tackle a wide variety of tasks without task-specific programming. Unlike their predecessors, these modern agents can break down complex problems into steps, use external tools when needed, and adapt to new situations they've never encountered before. This evolution represents a fundamental shift from automation to augmentation. Where early agents automated specific, predefined tasks, today's agents can understand our goals and work as collaborative partners in problem-solving. They can handle ambiguous instructions, incomplete information, and constantly changing contexts—capabilities that make them invaluable for building sophisticated applications like retrieval-augmented generation systems.

## Agents

When we talk about agents in 2025, we're entering a landscape where the term has become both ubiquitous and somewhat ambiguous. Different organizations and researchers use "agent" to describe everything from simple chatbots to fully autonomous systems that can operate independently for weeks. This diversity in definition isn't just academic—it reflects fundamentally different architectural approaches that will determine how we build the next generation of AI applications.

<img src="https://substackcdn.com/image/fetch/$s_!A_Oy!,f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2Fc3177e12-432e-4e41-814f-6febf7a35f68_1360x972.png" width=700>

At its core, an agent is a system that can perceive its environment, make decisions, and take actions to achieve specific goals. However, the way these capabilities are implemented varies dramatically. Some define agents as fully autonomous systems that operate independently over extended periods, using various tools and adapting their strategies based on feedback. Think of these like a personal assistant who can manage your entire schedule, book flights, handle emails, and make decisions on your behalf without constant supervision.

Others use the term more broadly to describe any system that follows predefined workflows to accomplish tasks. These implementations are more like following a detailed recipe—each step is predetermined, and while the system can handle some variations, it operates within clearly defined boundaries. The distinction between these approaches is crucial because it affects everything from system reliability to development complexity.

The most useful way to think about this spectrum is through the lens of control and decision-making. Workflows are systems where large language models and tools are orchestrated through predefined code paths. Every decision point is anticipated by the developer, and the system follows predetermined logic to handle different scenarios. Agents, in contrast, are systems where the LLM dynamically directs its own processes and tool usage, maintaining control over how it accomplishes tasks. The model itself decides what to do next, which tools to use, and how to adapt when things don't go as planned.

#### Simplicity defines perfectionism not complexity


When building applications with LLMs, the fundamental principle should be finding the simplest solution that meets your requirements. This might mean not building agentic systems at all. Agentic systems inherently trade latency and cost for better task performance, and you need to carefully consider when this tradeoff makes sense for your specific use case.

When more complexity is warranted, workflows offer predictability and consistency for well-defined tasks where you can anticipate most scenarios and edge cases. They're excellent for standardized processes like data processing pipelines, content moderation, or structured analysis tasks. Agents become the better choice when you need flexibility and model-driven decision-making at scale—situations where the variety of inputs and required responses is too broad to predefine, or where the system needs to adapt to entirely new scenarios.

The reality is that for many applications, the most effective approach involves optimizing single LLM calls with retrieval and in-context examples rather than building complex agentic systems. However, as we'll explore throughout this tutorial, there are compelling scenarios where the additional complexity of agents becomes not just beneficial, but necessary for achieving your goals. Understanding when and how to make this transition is what separates effective AI system builders from those who over-engineer solutions to problems that could be solved more simply.




#### Prompts


Prompts are the fundamental interface between human intent and AI capabilities, serving as the bridge that translates our natural language requests into structured instructions that language models can understand and act upon. In the context of agentic systems, prompts become even more critical because they not only convey what we want the agent to accomplish, but also how the agent should approach problem-solving, what tools it can use, and how it should reason through complex tasks.

Think of prompts as the instruction manual for your AI agent—just as a well-written manual can make the difference between a novice successfully assembling furniture or ending up with a pile of confused parts, a well-crafted prompt determines whether your agent performs brilliantly or struggles to understand your intent. The quality and structure of your prompts directly influence the agent's reasoning capabilities, tool usage patterns, and overall effectiveness in completing tasks.

<img src="https://www.datablist.com/_next/image?url=%2Fhowto_images%2Fhow-to-write-prompt-ai-agents%2Fstructured-ai-agent-prompt.png&w=3840&q=75" width=700>

There are several types of prompts that serve different purposes in agentic systems. System prompts establish the agent's role, personality, and fundamental operating principles—these are like giving someone their job description and company handbook before they start work. User prompts contain the specific tasks or questions you want the agent to handle, while few-shot prompts provide examples of desired input-output patterns to guide the agent's responses. Chain-of-thought prompts encourage step-by-step reasoning, helping agents break down complex problems into manageable pieces.

In multi-step agentic workflows, prompt engineering becomes particularly sophisticated because you need to design prompts that not only solve individual tasks but also coordinate between different stages of processing. The agent needs to understand when to use specific tools, how to interpret tool outputs, and how to maintain context across multiple interaction cycles. This requires careful consideration of prompt structure, token efficiency, and the logical flow of information through your system.

Let's explore how to implement basic prompt templates using LangChain with Google's Gemini model to see these concepts in action:

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os

# Initialize Gemini model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.7,
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

# Basic prompt template example
basic_template = PromptTemplate(
    input_variables=["topic", "audience"],
    template="""
    You are an expert educator who excels at explaining complex topics clearly.
    
    Topic: {topic}
    Audience: {audience}
    
    Please provide a clear, engaging explanation of this topic that is appropriate 
    for the specified audience. Include relevant examples and analogies to make 
    the concept accessible.
    """
)

# Create a simple chain
basic_chain = basic_template | llm | StrOutputParser()

# Test the basic template
response = basic_chain.invoke({
    "topic": "machine learning", 
    "audience": "high school students"
})
print("Basic Response:", response[:200] + "...")

In [ ]:
# Chat-based prompt template for more conversational interactions
chat_template = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful AI assistant with expertise in technology and science. 
    You provide accurate, clear explanations and can engage in detailed discussions.
    Always think step-by-step when solving problems and explain your reasoning."""),
    ("human", "I need help understanding {concept}. Can you break it down for me?"),
    ("ai", "I'd be happy to help explain {concept}! Let me break this down step by step."),
    ("human", "{user_question}")
])

# Create chat chain using LangChain's Expression Language (LCEL)
# This creates a composable pipeline: prompt -> model -> parser
# The | operator chains components where output of one becomes input of the next
chat_chain = chat_template | llm | StrOutputParser()

# What makes this powerful:
# 1. Declarative: Reads left-to-right like a data pipeline
# 2. Composable: Each component is modular and reusable
# 3. Streaming-ready: Automatically supports streaming responses
# 4. Type-safe: LangChain validates component compatibility
# 5. Async-compatible: Easy to convert to async execution

# Test chat template
chat_response = chat_chain.invoke({
    "concept": "neural networks",
    "user_question": "How do they actually learn from data?"
})
print("Chat Response:", chat_response[:300] + "...")

Great! now our LLM can respond to our questions, but how can we tweak it more to determine how much it weighs the prompt guideline while responding with it's own knowledge and reasoning? let's see!

### Hyperparameters

Hyperparameters are the control knobs that determine how a language model generates responses, acting like the settings on a sophisticated instrument that can dramatically change the output quality and behavior. Understanding these parameters is crucial for building effective agents because they directly influence how the model balances following prompt instructions versus drawing on its pre-trained knowledge, how creative or conservative its responses are, and how consistently it behaves across multiple interactions.

The most fundamental hyperparameter is **temperature**, which controls the randomness in the model's token selection process. Think of temperature like adjusting the creativity dial on the model's brain—at low temperatures (0.0-0.3), the model becomes highly deterministic, almost always choosing the most probable next token, resulting in consistent but potentially repetitive responses. At moderate temperatures (0.7-1.0), the model introduces controlled randomness, allowing for more creative and varied outputs while maintaining coherence. At high temperatures (1.5+), the model becomes highly unpredictable, often producing creative but potentially nonsensical text.

**Top-p (nucleus sampling)** works alongside temperature to refine token selection by considering only the smallest set of tokens whose cumulative probability exceeds the p threshold. For example, with top-p=0.9, the model only considers tokens that together account for 90% of the probability mass, effectively filtering out highly improbable options while maintaining diversity. This parameter is particularly important for maintaining quality while allowing creativity.

**Top-k** sets a hard limit on the number of highest-probability tokens to consider at each step. Unlike top-p's dynamic approach, top-k provides a fixed constraint—if k=40, only the 40 most likely tokens are considered regardless of their probability distribution. This can be useful for maintaining consistency in specialized domains where vocabulary should be limited.

**Max tokens** controls the maximum length of the generated response, serving as a computational and cost control mechanism. **Stop sequences** allow you to define specific strings that signal the model to cease generation, which is particularly useful in agentic workflows where you need precise control over output formatting.

Let's explore how these parameters affect model behavior in practice:

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Configurations to demonstrate hyperparameter effects
prompt = PromptTemplate(
    input_variables=["topic"],
    template="Explain {topic} in exactly three sentences. Be creative but accurate."
)

# Low temperature - deterministic, consistent responses
conservative_llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.1,  # Very low temperature for consistency
    max_tokens=150,
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

# Moderate temperature - balanced creativity and consistency  
balanced_llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.7,  # Standard temperature for most applications
    max_tokens=150,
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

# High temperature - creative, varied responses
creative_llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=1.2,  # Higher temperature for creativity
    max_tokens=150,
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

# Create chains for each configuration
conservative_chain = prompt | conservative_llm | StrOutputParser()
balanced_chain = prompt | balanced_llm | StrOutputParser()
creative_chain = prompt | creative_llm | StrOutputParser()

# Test the same prompt with different temperature settings
topic = "quantum computing"

print("=== CONSERVATIVE (Temperature=0.1) ===")
conservative_response = conservative_chain.invoke({"topic": topic})
print(conservative_response)

print("\n=== BALANCED (Temperature=0.7) ===")
balanced_response = balanced_chain.invoke({"topic": topic})
print(balanced_response)

print("\n=== CREATIVE (Temperature=1.2) ===")
creative_response = creative_chain.invoke({"topic": topic})
print(creative_response)

In [ ]:
#  how hyperparameters affect prompt adherence vs. knowledge utilization
instruction_following_prompt = PromptTemplate(
    input_variables=["format", "content"],
    template="""
    You must follow this format EXACTLY: {format}
    
    Content to format: {content}
    
    Remember: Strict adherence to the format is required.
    """
)

# High instruction-following (low temperature)
strict_llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.0,  # Maximum determinism for format adherence
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

# More flexible interpretation (higher temperature)
flexible_llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro", 
    temperature=0.9,  # More creativity, less strict adherence
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

strict_chain = instruction_following_prompt | strict_llm | StrOutputParser()
flexible_chain = instruction_following_prompt | flexible_llm | StrOutputParser()

format_instruction = "1. [Topic] 2. [Definition] 3. [Example]"
content = "Machine learning algorithms that can improve automatically through experience"

print("=== STRICT ADHERENCE (Temperature=0.0) ===")
strict_result = strict_chain.invoke({
    "format": format_instruction,
    "content": content
})
print(strict_result)

print("\n=== FLEXIBLE INTERPRETATION (Temperature=0.9) ===") 
flexible_result = flexible_chain.invoke({
    "format": format_instruction,
    "content": content
})
print(flexible_result)

The examples above demonstrate how hyperparameters create a fundamental tradeoff between instruction following and creative knowledge application. Low temperature models excel at following precise formatting requirements and maintaining consistency across multiple calls, making them ideal for structured data extraction, API responses, and workflows where predictability is paramount. Higher temperature models bring more of the model's training knowledge into play, generating more diverse responses and creative solutions, but at the cost of strict instruction adherence.

This balance becomes critical in agentic systems where you need to decide whether your agent should be a precise executor of specific instructions or a creative problem-solver that can adapt its approach based on context. The choice often depends on your use case: customer service bots might need low-temperature consistency, while creative writing assistants might benefit from higher-temperature diversity.

Now that we understand how to control our model's behavior through prompts and hyperparameters, we need to give our agents the ability to extend beyond their base knowledge and interact with the world. This is where tools come into play - they're what transform a language model from a sophisticated text generator into an active agent that can perform real actions and access current information.

### Tools



Tools are what transform language models from sophisticated text generators into active agents capable of performing real-world actions and accessing live information. Think of tools as the hands and senses of your AI agent - without them, even the most advanced language model is limited to working with only the knowledge it was trained on, which becomes stale the moment training ends. Tools bridge this gap by allowing agents to interact with databases, APIs, web services, file systems, and any other external systems your application needs to work with.

<img src="https://media.licdn.com/dms/image/v2/D4D12AQGyFCaSY8w4Ag/article-cover_image-shrink_720_1280/B4DZYg8dDRHAAI-/0/1744309441965?e=1762992000&v=beta&t=NS3gCnYSTWkxVwnRpHX6tCG7wcXcGgEknNpowIVAo2k" width=700>

The fundamental concept behind tools in agentic systems is function calling (also known as tool calling). Modern language models like GPT-4, Claude, and Gemini have been specifically trained to understand when they need external information or capabilities, and can generate structured function calls with appropriate parameters. When an agent encounters a question about current weather, stock prices, or needs to perform calculations, it doesn't hallucinate an answer - instead, it recognizes the limitation and calls the appropriate tool.

The tool execution process follows a predictable pattern: the agent receives a user request, analyzes what information or actions are needed, determines which tools to use, formats the tool calls with proper parameters, executes the tools, receives the results, and then synthesizes a response using both its knowledge and the tool outputs. This creates a powerful feedback loop where agents can chain multiple tool calls together, use the output of one tool as input to another, and dynamically adapt their approach based on intermediate results.

There are three main categories of tools we'll explore: **built-in tools** that come pre-integrated with language model providers, **explicit tools** that you define and implement yourself, and **Model Context Protocol (MCP) tools** that provide standardized interfaces for complex integrations. Each category serves different purposes and offers varying levels of customization and complexity.

#### Built-in Tools



Built-in tools are native capabilities provided directly by language model providers, eliminating the need for external integrations or custom implementations. Google's Gemini models, for example, come with several powerful built-in tools including Google Search integration, code execution capabilities, and mathematical computation tools. These tools are particularly valuable because they're optimized for the specific model, have minimal latency overhead, and don't require additional API keys or setup beyond your primary model access.

The advantage of built-in tools is their seamless integration - the model provider handles all the complexity of tool execution, result formatting, and error handling. When you enable Google Search for Gemini, the model can perform web searches and incorporate real-time information directly into its responses without any additional code on your part. Similarly, the code execution tool allows Gemini to write and run Python code in a sandboxed environment, making it excellent for data analysis, mathematical calculations, and generating visualizations.

Let's explore how to use Gemini's built-in tools with LangChain:

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os

# Initialize Gemini with built-in Google Search tool
# The 'google_search_retrieval' tool allows the model to search the web for current information
llm_with_search = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.3,
    google_api_key=os.getenv("GOOGLE_API_KEY"),
    # Enable Google Search integration - this gives the model access to real-time web information
    tools=["google_search_retrieval"]
)

# Create a prompt that would benefit from real-time information
search_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can search for current information when needed."),
    ("human", "{query}")
])

search_chain = search_prompt | llm_with_search | StrOutputParser()

# Test with a query that requires current information
current_info_query = "What are the latest developments in AI safety research in 2024?"
print("=== Query requiring current information ===")
search_response = search_chain.invoke({"query": current_info_query})
print(search_response[:500] + "...")

In [ ]:
# Initialize Gemini with code execution capability
# The 'code_execution' tool allows the model to write and run Python code
llm_with_code = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.1,  # Lower temperature for more reliable code execution
    google_api_key=os.getenv("GOOGLE_API_KEY"),
    # Enable code execution - model can write and run Python code in sandboxed environment
    tools=["code_execution"]
)

code_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a data analyst assistant. When you need to perform calculations or data analysis, write and execute Python code."),
    ("human", "{analysis_request}")
])

code_chain = code_prompt | llm_with_code | StrOutputParser()

# Test with a request that benefits from code execution
analysis_request = """
Analyze the following dataset and provide insights:
Sales data: [120, 150, 180, 95, 200, 175, 160, 140, 190, 210]

Calculate the mean, median, standard deviation, and identify any outliers.
Create a simple visualization if possible.
"""

print("=== Analysis with code execution ===")
code_response = code_chain.invoke({"analysis_request": analysis_request})
print(code_response)

In [ ]:
# Combining multiple built-in tools for comprehensive analysis
llm_with_multiple_tools = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.4,
    google_api_key=os.getenv("GOOGLE_API_KEY"),
    # Enable both search and code execution tools
    tools=["google_search_retrieval", "code_execution"]
)

comprehensive_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a research analyst assistant with access to both web search and code execution.
    Use web search to find current information and code execution for any calculations or data analysis.
    Always explain which tool you're using and why."""),
    ("human", "{research_question}")
])

comprehensive_chain = comprehensive_prompt | llm_with_multiple_tools | StrOutputParser()

# Test with a complex query that benefits from both tools
complex_query = """
Research the current market capitalization of the top 3 AI companies in 2024, 
then calculate what percentage each represents of the total combined market cap.
Show your calculations step by step.
"""

print("=== Research with multiple built-in tools ===")
comprehensive_response = comprehensive_chain.invoke({"research_question": complex_query})
print(comprehensive_response[:800] + "...")

#### Explicit Tools



While built-in tools provide excellent out-of-the-box functionality, the real power of agentic systems comes from creating custom tools tailored to your specific use case. Explicit tools are functions you define and implement yourself, giving agents the ability to interact with your databases, APIs, business logic, or any other systems your application requires. This is where agents transform from general-purpose assistants into specialized experts for your domain.

The process of creating explicit tools involves defining the tool's interface (what parameters it accepts and what it returns), implementing the actual functionality, and then registering the tool with your agent framework. LangChain makes this process straightforward through its `@tool` decorator and `Tool` class, which handle the integration details while letting you focus on the business logic.

<img src="https://miro.medium.com/v2/resize:fit:2000/1*fu9Lu8D8DLnVFPAWg7N0jQ.png" width=700>

When designing explicit tools, it's important to think about granularity and composability. Rather than creating one massive tool that does everything, it's better to create focused tools that do one thing well and can be combined. For example, instead of a single "manage_database" tool, you might create separate "query_user", "update_inventory", and "calculate_metrics" tools that can work together.

Let's explore how to create and use explicit tools with LangChain and Gemini:

In [ ]:
from langchain_core.tools import tool
from langchain.tools import Tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
import json
import random
from typing import List, Dict
import datetime

# Define custom tools using the @tool decorator
# This decorator automatically handles the tool registration and parameter validation

@tool
def get_weather(city: str, country: str = "US") -> str:
    """
    Get current weather information for a specified city.
    
    Args:
        city: The name of the city to get weather for
        country: The country code (default: US)
    
    Returns:
        JSON string with weather information
    """
    # Simulate weather API call - in practice, this would call a real weather service
    weather_conditions = ["sunny", "cloudy", "rainy", "snowy", "partly cloudy"]
    temperature = random.randint(-10, 35)
    condition = random.choice(weather_conditions)
    
    weather_data = {
        "city": city,
        "country": country,
        "temperature": temperature,
        "condition": condition,
        "humidity": random.randint(30, 90),
        "timestamp": datetime.datetime.now().isoformat()
    }
    
    return json.dumps(weather_data, indent=2)

@tool
def calculate_compound_interest(principal: float, rate: float, time: int, compounds_per_year: int = 1) -> str:
    """
    Calculate compound interest for investment planning.
    
    Args:
        principal: Initial investment amount
        rate: Annual interest rate (as decimal, e.g., 0.05 for 5%)
        time: Number of years
        compounds_per_year: How many times interest compounds per year (default: 1)
    
    Returns:
        Formatted string with calculation details
    """
    # A = P(1 + r/n)^(nt)
    amount = principal * (1 + rate/compounds_per_year) ** (compounds_per_year * time)
    interest_earned = amount - principal
    
    result = {
        "principal": principal,
        "annual_rate": f"{rate*100}%",
        "time_years": time,
        "compounds_per_year": compounds_per_year,
        "final_amount": round(amount, 2),
        "interest_earned": round(interest_earned, 2),
        "total_return_percentage": round((interest_earned/principal)*100, 2)
    }
    
    return json.dumps(result, indent=2)

@tool  
def search_user_database(query: str, user_type: str = "all") -> str:
    """
    Search a simulated user database for customer information.
    
    Args:
        query: Search term (name, email, or ID)
        user_type: Filter by user type - "premium", "basic", or "all" (default)
    
    Returns:
        JSON string with user information
    """
    # Simulate database search - in practice, this would query your actual database
    mock_users = [
        {"id": "001", "name": "Alice Johnson", "email": "alice@email.com", "type": "premium", "status": "active"},
        {"id": "002", "name": "Bob Smith", "email": "bob@email.com", "type": "basic", "status": "active"}, 
        {"id": "003", "name": "Carol Davis", "email": "carol@email.com", "type": "premium", "status": "inactive"},
        {"id": "004", "name": "David Wilson", "email": "david@email.com", "type": "basic", "status": "active"}
    ]
    
    # Filter by user type if specified
    if user_type != "all":
        mock_users = [user for user in mock_users if user["type"] == user_type]
    
    # Search logic
    results = []
    query_lower = query.lower()
    for user in mock_users:
        if (query_lower in user["name"].lower() or 
            query_lower in user["email"].lower() or 
            query_lower == user["id"]):
            results.append(user)
    
    return json.dumps({"query": query, "results": results}, indent=2)

# Collect all tools in a list
custom_tools = [get_weather, calculate_compound_interest, search_user_database]

print("=== Custom Tools Defined ===")
print(f"Created {len(custom_tools)} custom tools:")
for tool in custom_tools:
    print(f"- {tool.name}: {tool.description}")

In [ ]:
# Create an agent that can use our custom tools
# We'll use Gemini as the underlying LLM with our custom tools

# Initialize Gemini for tool calling
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.1,  # Lower temperature for more reliable tool usage
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

# Create a prompt template for our tool-using agent
tool_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant with access to several tools:
    1. get_weather: Get current weather for any city
    2. calculate_compound_interest: Calculate investment returns
    3. search_user_database: Look up customer information
    
    Use these tools when needed to provide accurate, helpful responses.
    Always explain which tool you're using and why.
    If a tool returns JSON data, format it nicely for the user."""),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Create the tool-calling agent
# This agent will automatically decide when and how to use our custom tools
agent = create_tool_calling_agent(llm, custom_tools, tool_prompt)

# Create an agent executor to run the agent with tools
agent_executor = AgentExecutor(
    agent=agent, 
    tools=custom_tools, 
    verbose=True,  # Show the agent's thought process
    handle_parsing_errors=True  # Gracefully handle any parsing issues
)

print("=== Tool-Using Agent Created ===")
print("Agent ready with custom tools integrated")

In [ ]:
# Test the agent with different types of requests

print("=== Test 1: Weather Information ===")
weather_response = agent_executor.invoke({
    "input": "What's the weather like in Tokyo, Japan right now?"
})
print("Response:", weather_response['output'])

print("\n=== Test 2: Financial Calculation ===")
investment_response = agent_executor.invoke({
    "input": "If I invest $10,000 at 6% annual interest compounded monthly for 10 years, how much will I have?"
})
print("Response:", investment_response['output'])

print("\n=== Test 3: Database Search ===")
user_search_response = agent_executor.invoke({
    "input": "Can you find information about user Alice in our database?"
})
print("Response:", user_search_response['output'])

print("\n=== Test 4: Multi-tool Chain ===")
complex_response = agent_executor.invoke({
    "input": """I'm planning a trip to San Francisco and want to:
    1. Check the weather there
    2. Find any premium users in our database named David
    3. Calculate how much $5000 invested at 4.5% annual interest for 5 years would grow to
    
    Please help with all three requests."""
})
print("Response:", complex_response['output'])

#### Model Context Protocol (MCP)



Model Context Protocol (MCP) represents the next evolution in AI tool integration, providing a standardized way for AI applications to securely connect to data sources and tools. Think of MCP as a universal translator that allows any AI system to communicate with any external service through a common protocol, eliminating the need for custom integrations for each tool or data source.

<img src="https://pbs.twimg.com/tweet_video_thumb/Gl7C44tXYAAdDSJ.jpg" width=700>

MCP was developed by Anthropic to solve the fragmentation problem in AI tool ecosystems. Before MCP, every AI application had to implement its own custom integrations for databases, APIs, file systems, and other external resources. This led to duplicated effort, security inconsistencies, and tools that only worked with specific AI platforms. MCP standardizes these interactions through a client-server architecture where MCP servers expose resources (like databases or file systems) and tools (like calculators or API clients) through a uniform interface.

The protocol operates on JSON-RPC 2.0, enabling real-time, bidirectional communication between AI applications (MCP clients) and external resources (MCP servers). This means your agent can not only call tools but also receive real-time updates, notifications, and streaming data from external systems. The security model is built around explicit capability declarations and sandboxed execution, ensuring that agents can only access resources they've been explicitly granted permission to use.

What makes MCP particularly powerful for RAG and agentic systems is its ability to provide **contextual data access**. Instead of just calling functions, MCP servers can expose rich contextual information about resources - like database schemas, file structures, or API capabilities - allowing agents to make more informed decisions about how to interact with external systems.

Let's explore how to integrate MCP servers with LangChain and Gemini. For this example, we'll use the MCP SDK to create a simple server and then connect to it:

In [ ]:
# Note: This example demonstrates MCP concepts. In practice, you would install:
# pip install mcp langchain-mcp

# For now, we'll simulate MCP functionality to understand the concepts
from typing import Any, Dict, List
import json
import asyncio
from dataclasses import dataclass

# Simulate an MCP server interface
@dataclass
class MCPResource:
    """Represents a resource exposed by an MCP server"""
    uri: str
    name: str
    description: str
    mime_type: str

@dataclass 
class MCPTool:
    """Represents a tool exposed by an MCP server"""
    name: str
    description: str
    input_schema: Dict[str, Any]

class MockMCPServer:
    """Simulated MCP server for demonstration purposes"""
    
    def __init__(self, name: str):
        self.name = name
        self.resources: List[MCPResource] = []
        self.tools: List[MCPTool] = []
        
    def add_resource(self, resource: MCPResource):
        self.resources.append(resource)
        
    def add_tool(self, tool: MCPTool):
        self.tools.append(tool)
        
    def list_resources(self) -> List[Dict[str, Any]]:
        """List all available resources"""
        return [
            {
                "uri": r.uri,
                "name": r.name, 
                "description": r.description,
                "mimeType": r.mime_type
            } for r in self.resources
        ]
        
    def list_tools(self) -> List[Dict[str, Any]]:
        """List all available tools"""
        return [
            {
                "name": t.name,
                "description": t.description,
                "inputSchema": t.input_schema
            } for t in self.tools
        ]
        
    def read_resource(self, uri: str) -> str:
        """Read content from a resource"""
        # Simulate resource reading
        if "customer_db" in uri:
            return json.dumps({
                "customers": [
                    {"id": 1, "name": "John Doe", "email": "john@example.com", "tier": "gold"},
                    {"id": 2, "name": "Jane Smith", "email": "jane@example.com", "tier": "silver"}
                ],
                "schema": {
                    "id": "integer",
                    "name": "string", 
                    "email": "string",
                    "tier": "string"
                }
            })
        elif "inventory" in uri:
            return json.dumps({
                "items": [
                    {"sku": "A001", "name": "Laptop", "quantity": 50, "price": 999.99},
                    {"sku": "A002", "name": "Mouse", "quantity": 200, "price": 29.99}
                ]
            })
        return "Resource not found"
        
    def call_tool(self, tool_name: str, arguments: Dict[str, Any]) -> str:
        """Execute a tool with given arguments"""
        if tool_name == "query_analytics":
            metric = arguments.get("metric", "sales")
            period = arguments.get("period", "month")
            return json.dumps({
                "metric": metric,
                "period": period,
                "value": 150000 if metric == "sales" else 1200,
                "trend": "increasing",
                "timestamp": "2024-10-22T10:00:00Z"
            })
        elif tool_name == "send_notification":
            return json.dumps({
                "status": "sent",
                "recipient": arguments.get("recipient"),
                "message": arguments.get("message"),
                "delivery_id": "notify_12345"
            })
        return json.dumps({"error": "Tool not found"})

# Create a mock MCP server with business resources and tools
business_mcp = MockMCPServer("business_system")

# Add resources (data sources the agent can read)
business_mcp.add_resource(MCPResource(
    uri="mcp://business/customer_db",
    name="Customer Database",
    description="Customer information and account details", 
    mime_type="application/json"
))

business_mcp.add_resource(MCPResource(
    uri="mcp://business/inventory",
    name="Inventory System", 
    description="Product inventory and stock levels",
    mime_type="application/json"
))

# Add tools (actions the agent can perform)
business_mcp.add_tool(MCPTool(
    name="query_analytics",
    description="Query business analytics and metrics",
    input_schema={
        "type": "object",
        "properties": {
            "metric": {"type": "string", "enum": ["sales", "users", "revenue"]},
            "period": {"type": "string", "enum": ["day", "week", "month", "year"]}
        },
        "required": ["metric"]
    }
))

business_mcp.add_tool(MCPTool(
    name="send_notification", 
    description="Send notifications to users or systems",
    input_schema={
        "type": "object",
        "properties": {
            "recipient": {"type": "string"},
            "message": {"type": "string"},
            "priority": {"type": "string", "enum": ["low", "medium", "high"]}
        },
        "required": ["recipient", "message"]
    }
))

print("=== MCP Server Created ===")
print(f"Server: {business_mcp.name}")
print(f"Resources: {len(business_mcp.resources)}")
print(f"Tools: {len(business_mcp.tools)}")

# List available resources and tools
print("\n=== Available Resources ===")
for resource in business_mcp.list_resources():
    print(f"- {resource['name']}: {resource['description']}")
    
print("\n=== Available Tools ===") 
for tool in business_mcp.list_tools():
    print(f"- {tool['name']}: {tool['description']}")

In [ ]:
# Create LangChain tools that interface with our MCP server
# This demonstrates how MCP servers can be integrated into LangChain workflows

@tool
def mcp_read_resource(resource_name: str) -> str:
    """
    Read data from MCP server resources like databases or file systems.
    
    Args:
        resource_name: Name of the resource to read (customer_db, inventory)
    
    Returns:
        JSON string with resource data
    """
    uri_map = {
        "customer_db": "mcp://business/customer_db",
        "customers": "mcp://business/customer_db", 
        "inventory": "mcp://business/inventory",
        "products": "mcp://business/inventory"
    }
    
    uri = uri_map.get(resource_name.lower())
    if not uri:
        return json.dumps({"error": f"Resource '{resource_name}' not found"})
        
    return business_mcp.read_resource(uri)

@tool
def mcp_query_analytics(metric: str, period: str = "month") -> str:
    """
    Query business analytics through MCP server.
    
    Args:
        metric: The metric to query (sales, users, revenue)
        period: Time period for the metric (day, week, month, year)
    
    Returns:
        JSON string with analytics data
    """
    return business_mcp.call_tool("query_analytics", {
        "metric": metric,
        "period": period
    })

@tool  
def mcp_send_notification(recipient: str, message: str, priority: str = "medium") -> str:
    """
    Send notifications through MCP server.
    
    Args:
        recipient: Who to send the notification to
        message: The notification message
        priority: Priority level (low, medium, high)
    
    Returns:
        JSON string with delivery confirmation
    """
    return business_mcp.call_tool("send_notification", {
        "recipient": recipient,
        "message": message,
        "priority": priority
    })

# Create MCP-enabled tools list
mcp_tools = [mcp_read_resource, mcp_query_analytics, mcp_send_notification]

# Create an agent that can use MCP tools
mcp_llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.2,
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

mcp_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a business intelligence assistant with access to company systems through MCP.
    
    Available MCP resources:
    - customer_db: Customer information and account details
    - inventory: Product inventory and stock levels
    
    Available MCP tools:
    - mcp_query_analytics: Get business metrics and analytics
    - mcp_send_notification: Send notifications to users or systems
    - mcp_read_resource: Read data from company databases and systems
    
    Use these tools to provide comprehensive business insights and take actions when requested.
    Always format data nicely and explain what you're doing."""),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

mcp_agent = create_tool_calling_agent(mcp_llm, mcp_tools, mcp_prompt)
mcp_executor = AgentExecutor(
    agent=mcp_agent,
    tools=mcp_tools,
    verbose=True,
    handle_parsing_errors=True
)

print("=== MCP-Enabled Agent Created ===")
print("Agent ready with MCP server integration")

In [ ]:
# Test the MCP-enabled agent with business scenarios

print("=== Test 1: Customer Data Analysis ===")
customer_analysis = mcp_executor.invoke({
    "input": "Can you analyze our customer data? I want to see the customer information and understand our customer tiers."
})
print("Response:", customer_analysis['output'])

print("\n=== Test 2: Business Analytics ===")
analytics_query = mcp_executor.invoke({
    "input": "What were our sales metrics for this month? Also check user metrics."
})
print("Response:", analytics_query['output'])

print("\n=== Test 3: Inventory Management ===") 
inventory_check = mcp_executor.invoke({
    "input": "Check our current inventory levels and identify any products that might need restocking."
})
print("Response:", inventory_check['output'])

print("\n=== Test 4: Complex Business Workflow ===")
complex_workflow = mcp_executor.invoke({
    "input": """I need a comprehensive business report:
    1. Check our customer database for gold tier customers
    2. Get our current sales metrics
    3. Review inventory levels
    4. If sales are good and we have low inventory, send a notification to 'inventory-team@company.com' about restocking
    
    Please provide a summary with actionable insights."""
})
print("Response:", complex_workflow['output'])

The examples above demonstrate the power of tools in transforming language models into capable agents. We've seen how **built-in tools** provide immediate capabilities with minimal setup, **explicit tools** offer complete customization for your specific needs, and **MCP tools** enable standardized integration with complex systems while maintaining security and scalability.

The key insight is that tools are what bridge the gap between language model intelligence and real-world utility. Without tools, even the most sophisticated language model is limited to generating text based on its training data. With tools, agents become active participants in your business processes, capable of querying databases, performing calculations, calling APIs, and taking actions in response to user needs.

As we design agentic systems, the choice between different tool types depends on your specific requirements:
- Use **built-in tools** when the model provider offers functionality that meets your needs
- Create **explicit tools** when you need custom integration with your specific systems  
- Implement **MCP tools** when you need standardized, scalable integrations across multiple AI applications

Now that our agents can take actions in the world through tools, we need to ensure they can maintain context and remember information across interactions. This is where memory and context management become crucial for building agents that can handle complex, multi-step workflows and maintain coherent conversations over time.

### Context Management

<img src="https://substackcdn.com/image/fetch/$s_!AyLS!,f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2Fb0e3c002-0841-4d5f-9171-3eb63c321824_1600x1224.png" width=700>

##### LangChain ConversationBufferMemory

##### LangChain ConversationSummaryMemory


##### LangChain ConversationBufferWindowMemory


##### LangChain ConversationTokenBufferMemory


##### LangGraph MemorySaver


##### LangGraph MessagesState


##### LlamaIndex ChatMemoryBuffer








##### LlamaIndex VectorMemory

#### Skills

### Workflows and Chains



##### Parallelization Workflow



##### LangGraph Parallel Execution


##### LangChain RunnableParallel

##### LangGraph Multi-Agent Patterns


##### LangChain Agent Executors


##### LlamaIndex AgentRunner

## RAG

### But Why RAG?

Talk about LLM system in general, while introducing agents, where those workflows lack the limit of llms in context and actions

### Finding the Data

#### Webscraping


##### LangChain WebBaseLoader


##### LangChain AsyncHtmlLoader


##### LangChain SitemapLoader


##### LangChain PlaywrightURLLoader


##### LlamaIndex SimpleWebPageReader



##### LlamaIndex BeautifulSoupWebReader

#### Document Loading


##### LangChain PyPDFLoader


##### LangChain UnstructuredFileLoader


##### LangChain CSVLoader


##### LangChain JSONLoader


##### LlamaIndex SimpleDirectoryReader


##### LlamaIndex PDFReader

### Preprocessing the documents

#### Splitting


##### LangChain RecursiveCharacterTextSplitter


##### LangChain TokenTextSplitter


##### LangChain MarkdownHeaderTextSplitter


##### LangChain PythonCodeTextSplitter


##### LlamaIndex SentenceSplitter


##### LlamaIndex SemanticSplitterNodeParser


##### LlamaIndex HierarchicalNodeParser

#### Chunking



##### LangChain SemanticChunker


##### LangChain ParentDocumentRetriever


##### LlamaIndex SimpleNodeParser


##### LlamaIndex SentenceWindowNodeParser

#### Embedding


##### LangChain OpenAIEmbeddings


##### LangChain HuggingFaceEmbeddings


##### LlamaIndex OpenAIEmbedding



##### LlamaIndex HuggingFaceEmbedding

### Storing Documents

#### Vector Databases


##### LangChain Chroma Integration


##### LangChain Pinecone Integration


##### LangChain FAISS Integration


##### LlamaIndex ChromaVectorStore



##### LlamaIndex PineconeVectorStore

#### Knowledge Graphs


##### LangGraph StateGraph


##### LangChain Neo4jGraph



##### LlamaIndex KnowledgeGraphIndex

#### SQL


##### LangChain SQLDatabase


##### LangChain SQLDatabaseChain



##### LlamaIndex SQLStructStoreIndex

### Retrieval Mechanisms

#### Vector search


##### LangChain VectorStoreRetriever


##### LangChain MultiVectorRetriever


##### LlamaIndex VectorIndexRetriever



##### LlamaIndex VectorIndexAutoRetriever

#### Tree Search

#### Node Search

#### Hybrid Search

##### LangChain EnsembleRetriever
##### LangChain BM25Retriever
##### LlamaIndex QueryFusionRetriever

##### LangChain ConditionalEdge


##### LangGraph Router Patterns


##### LlamaIndex RouterQueryEngine

### Evaluation

#### Faithfulness & Accuracy

#### RAGAS (RAG Assessment)



##### LangSmith + RAGAS Integration


##### LangChain Evaluation Chains

#### TruLens RAG Triad


#### Multi-Agent Metrics


#### Advanced Agentic Patterns

#### Human Evaluation


#### LLM-as-Judge



##### LangSmith Tracing


##### LangSmith Evaluation Datasets


##### LangSmith Custom Evaluators

## A Complete Agentic System



##### LangGraph Agent Architecture


##### LangChain Agent Types (ReAct, Plan-and-Execute)


##### LangSmith Agent Monitoring


##### LlamaIndex Multi-Agent Orchestrator

## Limitations & Variations

#### RAPTOR

#### Self-RAG

#### CRAG

#### Adaptive RAG

## Summary

## Citations